# Gait detection
This notebook shows how to predict gait probability using extracted features from windowed accelerometry data as input.

## Modules

In [2]:
import numpy as np
import os
import pandas as pd
import tsdf

## Constans

In [ ]:
path_to_data_folder = '../../../tests/data'
path_to_classifier_input = os.path.join(path_to_data_folder, '4.predictions')
path_to_input_data = os.path.join(path_to_data_folder, '3.extracted_features')

classifier_file_name = 'clf_gait_detection.pkl'
thresholds_file_name = 'threshold_gait_detection.txt'

meta_filename = 'gait_meta.json'
time_filename = 'gait_time.bin'
values_filename = 'gait_values.bin'

## Load data

#### Features

In [ ]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(path_to_input_data, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]

df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df.head(2)

#### Classifier

In [ ]:
clf = pd.read_pickle(os.path.join(path_to_classifier_input, classifier_file_name))

with open(os.path.join(path_to_classifier_input, thresholds_file_name), 'r') as f:
    thresholds_str = f.read()

l_thresholds = [float(x) for x in thresholds_str.split(' ')]